<a href="https://colab.research.google.com/github/tsato-code/colab_notebooks/blob/main/hashibiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Apr 15 15:05:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 関連リポジトリをクローン
%cd /content
!git clone https://github.com/KaiyangZhou/vsumm-reinforce

/content
Cloning into 'vsumm-reinforce'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 641.05 KiB | 10.17 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
# データセットと学習済みモデルをダウンロード
%cd vsumm-reinforce
#!wget http://www.eecs.qmul.ac.uk/~kz303/vsumm-reinforce/datasets.tar.gz
#!tar -xvzf datasets.tar.gz
#!wget http://www.eecs.qmul.ac.uk/~kz303/vsumm-reinforce/models.tar.gz
#!tar -xvzf models.tar.gz

/content/vsumm-reinforce


In [4]:
%cd ..
# GoogLeNet for Image Classification
!git clone https://github.com/conan7882/GoogLeNet-Inception-tf
%cd GoogLeNet-Inception-tf
!wget https://www.dropbox.com/sh/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy
%cd ..
# Kernel Temporal Segmentation
!wget http://pascal.inrialpes.fr/data2/potapov/med_summaries/kts_ver1.1.tar.gz
!tar -zxvf kts_ver1.1.tar.gz

/content
Cloning into 'GoogLeNet-Inception-tf'...
remote: Enumerating objects: 332, done.
remote: Total 332 (delta 0), reused 0 (delta 0), pack-reused 332
Receiving objects: 100% (332/332), 11.78 MiB | 44.19 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/GoogLeNet-Inception-tf
--2021-04-15 15:06:10--  https://www.dropbox.com/sh/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy [following]
--2021-04-15 15:06:10--  https://www.dropbox.com/sh/raw/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc21529d8ad5cb9d295fc286d41b.dl.dropboxusercontent.

In [5]:
!mkdir -p data/original
%cd data/original

/content/data/original


In [10]:
!pip install pytube
from pytube import YouTube
import os

data_dir = '/content/data/original/'
# hot pepper beauty
out = YouTube("https://www.youtube.com/watch?v=rwt3oS_mFVQ").streams.first().download()
os.rename(out, 'test01.mp4')
# GREEN DAKARA
out = YouTube("https://www.youtube.com/watch?v=cShbQ2pQs94").streams.first().download()
os.rename(out, 'test02.mp4')
# ポカリスウェット
out = YouTube("https://www.youtube.com/watch?v=cKM2HQLK8Pg").streams.first().download()
os.rename(out, 'test03.mp4')

In [11]:
# スクリプトを修正
text = r'''
#!/bin/bash

"""
This script decompose a video into frames
How to use: replace path_to_videos and path_to_frames with real paths
"""

for f in /content/data/original/*.mp4
do
  echo "Processing $f file..."
  # take action on each file. $f store current file name\
  basename=$(ff=${f%.ext} ; echo ${ff##*/})
  name=$(echo $basename | cut -d'.' --complement -f2-)
  echo $f
 mkdir -p /content/data/frames/"$name"
 ffmpeg -i "$f" -f image2 -qscale:v 2 /content/data/frames/"$name"/%06d.jpg
done
'''
with open('/content/vsumm-reinforce/extra-tools/videos2frames.sh', 'w') as f:
    f.write(text)

In [12]:
# 修正したスクリプトを実行
%cd /content/vsumm-reinforce/extra-tools
!./videos2frames.sh

/content/vsumm-reinforce/extra-tools
./videos2frames.sh: line 7: $'\nThis script decompose a video into frames\nHow to use: replace path_to_videos and path_to_frames with real paths\n': command not found
Processing /content/data/original/test01.mp4 file...
/content/data/original/test01.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-li

In [13]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6MB 49kB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.2MB 37.2MB/s 
     |████████████████████████████████| 368kB 41.0MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [14]:
# 特徴量抽出のコード
text = r'''
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# File: inception_pretrained.py
# Author: Qian Ge <geqian1001@gmail.com>

import os
import sys
import platform
import argparse
import numpy as np
import tensorflow as tf

sys.path.append('../')
import loader as loader
from src.nets.googlenet import GoogLeNet
import h5py
import src.models.layers as L

# PRETRINED_PATH = '/home/qge2/workspace/data/pretrain/inception/googlenet.npy'
# DATA_PATH = '../data/'
PRETRINED_PATH = '/content/GoogLeNet-Inception-tf/googlenet.npy'
DATA_PATH = '/content/data/frames/test01/'
DATASET_PATH = '/content/vsumm-reinforce/datasets/eccv16_dataset_pokari_google_pool5.h5'
IM_CHANNEL = 3


def get_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--pretrained_path', type=str, default=PRETRINED_PATH,
                        help='Directory of pretrain model')
    parser.add_argument('--im_name', type=str, default='.jpg',
                        help='Part of image name')
    parser.add_argument('--data_path', type=str, default=DATA_PATH,
                        help='Directory of test images')
    
    return parser.parse_args()

def test_pre_trained():
    FLAGS = get_args()
    # Read ImageNet label into a dictionary
    label_dict = loader.load_label_dict()
    # Create a Dataflow object for test images
    image_data = loader.read_image(
        im_name=FLAGS.im_name, n_channel=IM_CHANNEL,
        data_dir=FLAGS.data_path, batch_size=1)

    # Create a testing GoogLeNet model
    test_model = GoogLeNet(
        n_channel=IM_CHANNEL, n_class=1000, pre_trained_path=FLAGS.pretrained_path)
    test_model.create_test_model()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        dataset = h5py.File(DATASET_PATH, 'w')
        video_name = FLAGS.data_path.split('/')[-2]
        features = np.empty((1, 1024), dtype=np.float64)  # Put dummy

        while image_data.epochs_completed < 1:
            # read batch files
            batch_data = image_data.next_batch_dict()
            # get batch file names
            batch_file_name = image_data.get_batch_file_name()[0]
            # get prediction results
            #pred = sess.run(test_model.layers['top_5'],
            feature = sess.run(L.global_avg_pool(test_model.layers['inception_out']),
                            feed_dict={test_model.image: batch_data['image']})
            features = np.concatenate((features, feature))

        dataset.create_dataset('{}/features_origin'.format(video_name), data=features[1:])
        dataset.close()

if __name__ == "__main__":
    test_pre_trained()
'''
with open('/content/GoogLeNet-Inception-tf/examples/feature_extraction.py', 'w') as f:
    f.write(text)

In [22]:
%cd /content/GoogLeNet-Inception-tf/examples

# これは一時的に必要なディレクトリ
!mkdir -p /content/vsumm-reinforce/datasets
# ディレクトリのパス設定次第でok
!python feature_extraction.py

/content/GoogLeNet-Inception-tf/examples
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:

In [16]:
text = r'''
import numpy as np
from cpd_auto import cpd_auto
import h5py

DATASET_PATH = "../vsumm-reinforce/datasets/eccv16_dataset_pokari_google_pool5.h5"
SAMP_RATE = 15  # sampling rate (fps)
N_CHANGE_POINTS = 30  # the number of change points

if __name__ == "__main__":
    dataset = h5py.File(DATASET_PATH, "r+")

    for video_name in dataset.keys():
        X = dataset[video_name]["features_origin"][...]
        n = X.shape[0]
        K = np.dot(X, X.T)
        cps, scores = cpd_auto(K, N_CHANGE_POINTS, 1)
        print("Estimated: (m=%d)" % len(cps), cps)

        cps = np.concatenate(([0], cps))
        cps = np.concatenate((cps, [n]))

        # Update dataset
        features = np.array([frame for i, frame in enumerate(X) if i % SAMP_RATE == 0], dtype=np.float64)
        change_points = np.array([[cps[i], cps[i + 1] - 1] for i in range(cps.shape[0] - 1)], dtype=np.int64)
        n_frames = np.array(n, dtype=np.int64)
        n_frame_per_seg = np.array([(p[1] - p[0] + 1) for p in change_points], dtype=np.int64)
        picks = np.array([(i * SAMP_RATE) for i in range((n - 1) // SAMP_RATE + 1)], dtype=np.int64)

        key = dataset[video_name]
        dataset.create_dataset(key + "/features", data=features)
        dataset.create_dataset(key + "/change_points", data=change_points)
        dataset.create_dataset(key + "/n_frames", data=n_frames)
        dataset.create_dataset(key + "/n_frame_per_seg", data=n_frame_per_seg)
        dataset.create_dataset(key + "/picks", data=picks)

    dataset.close()
'''
with open('/content/kts_ver1.1/segment.py', 'w') as f:
    f.write(text)

In [17]:
text = r'''
import numpy as np


def calc_scatters(K):
    """Calculate scatter matrix: scatters[i,j] = {scatter of the sequence with
    starting frame i and ending frame j}
    """
    n = K.shape[0]
    K1 = np.cumsum([0] + list(np.diag(K)))
    K2 = np.zeros((n + 1, n + 1))
    # TODO: use the fact that K - symmetric
    K2[1:, 1:] = np.cumsum(np.cumsum(K, 0), 1)

    diagK2 = np.diag(K2)

    i = np.arange(n).reshape((-1, 1))
    j = np.arange(n).reshape((1, -1))
    scatters = (
            K1[1:].reshape((1, -1)) - K1[:-1].reshape((-1, 1)) -
            (diagK2[1:].reshape((1, -1)) + diagK2[:-1].reshape((-1, 1)) -
             K2[1:, :-1].T - K2[:-1, 1:]) /
            ((j - i + 1).astype(np.float32) + (j == i - 1).astype(np.float32))
    )
    scatters[j < i] = 0

    return scatters


def cpd_nonlin(K, ncp, lmin=1, lmax=100000, backtrack=True, verbose=True,
               out_scatters=None):
    """Change point detection with dynamic programming
    :param K: Square kernel matrix
    :param ncp: Number of change points to detect (ncp >= 0)
    :param lmin: Minimal length of a segment
    :param lmax: Maximal length of a segment
    :param backtrack: If False - only evaluate objective scores (to save memory)
    :param verbose: If true, print verbose message
    :param out_scatters: Output scatters
    :return: Tuple (cps, obj_vals)
        - cps - detected array of change points: mean is thought to be constant
            on [ cps[i], cps[i+1] )
        - obj_vals - values of the objective function for 0..m changepoints
    """
    m = int(ncp)  # prevent numpy.int64

    n, n1 = K.shape
    assert n == n1, 'Kernel matrix awaited.'
    assert (m + 1) * lmin <= n <= (m + 1) * lmax
    assert 1 <= lmin <= lmax

    if verbose:
        print('Precomputing scatters...')
    J = calc_scatters(K)

    if out_scatters is not None:
        out_scatters[0] = J

    if verbose:
        print('Inferring best change points...')
    # I[k, l] - value of the objective for k change-points and l first frames
    I = 1e101 * np.ones((m + 1, n + 1))
    I[0, lmin:lmax] = J[0, lmin - 1:lmax - 1]

    if backtrack:
        # p[k, l] --- 'previous change' --- best t[k] when t[k+1] equals l
        p = np.zeros((m + 1, n + 1), dtype=int)
    else:
        p = np.zeros((1, 1), dtype=int)

    for k in range(1, m + 1):
        for l in range((k + 1) * lmin, n + 1):
            tmin = max(k * lmin, l - lmax)
            tmax = l - lmin + 1
            c = J[tmin:tmax, l - 1].reshape(-1) + \
                I[k - 1, tmin:tmax].reshape(-1)
            I[k, l] = np.min(c)
            if backtrack:
                p[k, l] = np.argmin(c) + tmin

    # Collect change points
    cps = np.zeros(m, dtype=int)

    if backtrack:
        cur = n
        for k in range(m, 0, -1):
            cps[k - 1] = p[k, cur]
            cur = cps[k - 1]

    scores = I[:, n].copy()
    scores[scores > 1e99] = np.inf
    return cps, scores
'''
with open('/content/kts_ver1.1/cpd_nonlin.py', 'w') as f:
    f.write(text)

In [31]:
%cd /content/kts_ver1.1
# python2との違いでエラーになってるっぽい。
# もしかしたらh5ファイルを読み込むところから書き換えが必要？
!python segment.py

/content/kts_ver1.1
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
Estimated: (m=30) [ 30  73  90 108 129 175 202 232 254 268 283 309 347 376 390 434 464 488
 504 531 572 660 664 668 673 680 690 698 701 704]


In [32]:
text = r'''
import theano
from theano import tensor as T

import theano_nets
from model_reinforceRNN import reinforceRNN

import numpy as np
from datetime import datetime
import time, math, os, sys, h5py, logging, vsum_tools, argparse
import os.path as osp
from scipy.spatial.distance import cdist

_DTYPE = theano.config.floatX
_PROPORTION = 0.10  # proportion to leave as a summary

def test(n_episodes=5,
         input_dim=1024,
         hidden_dim=256,
         W_init='normal',
         U_init='normal',
         weight_decay=1e-5,
         regularizer='L2',
         optimizer='adam',
         alpha=0.01,
         model_file='',
         eval_dataset='summe',
         verbose=True,
         ):
    # assert eval_dataset in ['summe', 'tvsum']
    assert os.path.isfile(model_file)

    if eval_dataset == 'summe':
        eval_metric = 'max'
    elif eval_dataset == 'tvsum':
        eval_metric = 'avg'
    model_options = locals().copy()

    log_dir = 'log-test'
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)

    logging.basicConfig(
      filename=log_dir+'/log.txt',
      filemode='w',
      format='%(asctime)s %(message)s',
      datefmt='[%d/%m/%Y %I:%M:%S]',
      level=logging.INFO
    )

    logger = logging.getLogger()
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    formatter = logging.Formatter(fmt='%(asctime)s %(message)s',datefmt='[%d/%m/%Y %I:%M:%S]')
    ch.setFormatter(formatter)
    logger.addHandler(ch)

    logger.info('initializing net model')
    net = reinforceRNN(model_options)

    logger.info('loading %s data' % (eval_dataset))
    h5f_path = 'datasets/eccv16_dataset_' + eval_dataset + '_google_pool5.h5'
    # dataset = h5py.File(h5f_path, 'r')
    dataset = h5py.File(h5f_path, 'r+')
    if sys.version_info[0] == 3:
        dataset_keys = list(dataset.keys())
    else:
        dataset_keys = dataset.keys()
    n_videos = len(dataset_keys)

    logger.info('=> testing')
    start_time = time.time()
    fms = []
    precs = []
    recs = []

    # save output results to h5 file
    # h5_res = h5py.File(osp.join(log_dir, 'result.h5'), 'w')

    for i_video in range(n_videos):
        key = dataset_keys[i_video]
        data_x = dataset[key]['features'][...].astype(_DTYPE)
        probs = net.model_inference(data_x)

        cps = dataset[key]['change_points'][...]
        n_frames = dataset[key]['n_frames'][()]
        nfps = dataset[key]['n_frame_per_seg'][...].tolist()
        positions = dataset[key]['picks'][...]

        # machine_summary = vsum_tools.generate_summary(probs, cps, n_frames, nfps, positions)
        machine_summary = vsum_tools.generate_summary(probs, cps, n_frames, nfps, positions, _PROPORTION)
        # user_summary = dataset[key]['user_summary'][...]
        # fm,prec,rec = vsum_tools.evaluate_summary(machine_summary, user_summary, eval_metric)
        # fms.append(fm)
        # precs.append(prec)
        # recs.append(rec)
        # if verbose: logger.info('video %s. fm=%f' % (key, fm))

        # save results for each test video
        # h5_res.create_dataset(key + '/score', data=probs)
        # h5_res.create_dataset(key + '/machine_summary', data=machine_summary)
        # h5_res.create_dataset(key + '/gtscore', data=dataset[key]['gtscore'][...])
        # h5_res.create_dataset(key + '/fm', data=fm)
        dataset.create_dataset(key + '/machine_summary', data=machine_summary)

    # h5_res.close()

    # mean_fm = np.mean(fms)
    # mean_prec = np.mean(precs)
    # mean_rec = np.mean(recs)

    logger.info('========================= conclusion =========================')
    logger.info('-- recap of model options')
    logger.info(str(model_options))
    # logger.info('-- final outcome')
    # logger.info('f-measure {:.1%}. precision {:.1%}. recall {:.1%}.'.format(mean_fm, mean_prec, mean_rec))
    elapsed_time = time.time() - start_time
    logger.info('elapsed time %.2f s' % (elapsed_time))
    logger.info('==============================================================')

    dataset.close()

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-model', type=str, default='', metavar='PATH')
    parser.add_argument('-d', type=str, default='tvsum')
    parser.add_argument('--in-dim', type=int, default=1024,
                        help="input dimension, i.e. dimension of CNN features")
    parser.add_argument('--h-dim', type=int, default=256,
                        help="hidden dimension of RNN")
    parser.add_argument('--verbose', action='store_true')

    args = parser.parse_args()

    test(input_dim=args.in_dim,
         hidden_dim=args.h_dim,
         model_file=args.model,
         eval_dataset=args.d,
         verbose=args.verbose)
'''
with open('/content/vsumm-reinforce/vsum_gen.py', 'w') as f:
    f.write(text)

In [ ]:
text = r'''
import numpy as np

"""
------------------------------------------------
Use dynamic programming (DP) to solve 0/1 knapsack problem
Time complexity: O(nW), where n is number of items and W is capacity

Author: Kaiyang Zhou
Website: https://kaiyangzhou.github.io/
------------------------------------------------
knapsack_dp(values,weights,n_items,capacity,return_all=False)

Input arguments:
  1. values: a list of numbers in either int or float, specifying the values of items
  2. weights: a list of int numbers specifying weights of items
  3. n_items: an int number indicating number of items
  4. capacity: an int number indicating the knapsack capacity
  5. return_all: whether return all info, defaulty is False (optional)

Return:
  1. picks: a list of numbers storing the positions of selected items
  2. max_val: maximum value (optional)
------------------------------------------------
"""
def knapsack_dp(values,weights,n_items,capacity,return_all=False):
    check_inputs(values,weights,n_items,capacity)

    table = np.zeros((n_items+1,capacity+1),dtype=np.float32)
    keep = np.zeros((n_items+1,capacity+1),dtype=np.float32)

    for i in xrange(1,n_items+1):
        for w in xrange(0,capacity+1):
            wi = weights[i-1] # weight of current item
            vi = values[i-1] # value of current item
            if (wi <= w) and (vi + table[i-1,w-wi] > table[i-1,w]):
                table[i,w] = vi + table[i-1,w-wi]
                keep[i,w] = 1
            else:
                table[i,w] = table[i-1,w]

    picks = []
    K = capacity

    for i in xrange(n_items,0,-1):
        if keep[i,K] == 1:
            picks.append(i)
            K -= weights[i-1]

    picks.sort()
    picks = [x-1 for x in picks] # change to 0-index

    if return_all:
        max_val = table[n_items,capacity]
        return picks,max_val
    return picks

def check_inputs(values,weights,n_items,capacity):
    # check variable type
    assert(isinstance(values,list))
    assert(isinstance(weights,list))
    assert(isinstance(n_items,int))
    assert(isinstance(capacity,int))
    # check value type
    assert(all(isinstance(val,int) or isinstance(val,float) for val in values))
    assert(all(isinstance(val,int) for val in weights))
    # check validity of value
    assert(all(val >= 0 for val in weights))
    assert(n_items > 0)
    assert(capacity > 0)

if __name__ == '__main__':
    values = [2,3,4]
    weights = [1,2,3]
    n_items = 3
    capacity = 3
    picks = knapsack_dp(values,weights,n_items,capacity)
    print(picks)
'''
with open('/content/vsumm-reinforce/knapsack.py', 'w') as f:
    f.write(text)

In [42]:
!pip install gdown
!gdown "https://drive.google.com/uc?export=download&id=1Bf0beMN_ieiM3JpprghaoOwQe9QJIyAN" \
    -O /content/vsumm-reinforce/datasets/datasets.zip
!unzip /content/vsumm-reinforce/datasets/datasets.zip

Downloading...
From: https://drive.google.com/uc?export=download&id=1Bf0beMN_ieiM3JpprghaoOwQe9QJIyAN
To: /content/vsumm-reinforce/datasets/datasets.zip
173MB [00:01, 98.0MB/s]
Archive:  /content/vsumm-reinforce/datasets/datasets.zip
  inflating: datasets/eccv16_dataset_ovp_google_pool5.h5  
  inflating: datasets/eccv16_dataset_tvsum_google_pool5.h5  
  inflating: datasets/eccv16_dataset_summe_google_pool5.h5  
  inflating: datasets/readme.txt     
  inflating: datasets/eccv16_dataset_youtube_google_pool5.h5  


In [46]:
# numpyのエラー？
# .astype('float32')が必要？？
!python vsum_train.py \
    --dataset datasets/eccv16_dataset_tvsum_google_pool5.h5 \
    --max-epochs 60 \
    --hidden-dim 256

[15/04/2021 04:04:26] model options: {'n_episodes': 5, 'input_dim': 1024, 'hidden_dim': 256, 'W_init': 'normal', 'U_init': 'normal', 'weight_decay': 1e-05, 'regularizer': 'L2', 'optimizer': 'adam', 'base_lr': 1e-05, 'decay_rate': 0.1, 'max_epochs': 60, 'decay_stepsize': -1, 'ignore_distant_sim': False, 'distant_sim_thre': 20, 'alpha': 0.01, 'model_file': None, 'disp_freq': 1, 'train_dataset_path': 'datasets/eccv16_dataset_tvsum_google_pool5.h5'}
[15/04/2021 04:04:26] initializing net model
Traceback (most recent call last):
  File "vsum_train.py", line 158, in <module>
    train_dataset_path=args.dataset)
  File "vsum_train.py", line 56, in train
    net = reinforceRNN(model_options)
  File "/content/vsumm-reinforce/model_reinforceRNN.py", line 40, in __init__
    init_state=None, init_memory=None, go_backwards=False
  File "/content/vsumm-reinforce/theano_nets.py", line 342, in __init__
    self.output = self.step(self.state_below)
  File "/content/vsumm-reinforce/theano_nets.py", lin

In [37]:
%cd /content/vsumm-reinforce
!THEANO_FLAGS='floatX=float32,device=cuda0' 
!python vsum_gen.py -model models/model_tvsum_reinforceRNN.h5 -d test01

/content/vsumm-reinforce
Traceback (most recent call last):
  File "vsum_gen.py", line 139, in <module>
    verbose=args.verbose)
  File "vsum_gen.py", line 31, in test
    assert os.path.isfile(model_file)
AssertionError


In [ ]:
# モデルファイルがないし、つくれないぽい
# 今のところ、エラー